In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [ ]:
'''
LOADING DATASET
'''
train = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),  # Normalize the image to [0-1] from [0-255]
                            download=True)

test = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())


In [ ]:
print(len(train))

60000


In [ ]:
from torch import tensor
traindata = [train[i] for i in range(len(train))]
train = torch.stack([d[0] for d in traindata], dim=0)
train=train[0:59999]
ys = [d[1] for d in traindata]
train_y = tensor(ys)

testdata = [test[i] for i in range(len(test))]
test = torch.stack([d[0] for d in testdata], dim=0)
test=test[0:9999]
ys = [d[1] for d in testdata]
test_y = tensor(ys)

# Seeding

In [ ]:
torch.manual_seed(120)

# Dataset Loading

In [ ]:
train = torch.rand(6000, 28, 28, 1)
test = torch.rand(1000, 28, 28, 1)
train_y = torch.randint(0,9, (6000,))
test_y = torch.randint(0,9, (1000,))

In [ ]:
print(train_y)

tensor([1, 4, 5,  ..., 2, 8, 0])


# Model Architeture

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=16,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=5, stride=1, padding=2)

        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2,2)

        self.linear_layer_1 = nn.Linear(32*7*7, 512)
        self.linear_layer_2 = nn.Linear(512, 128)
        self.linear_layer_3 = nn.Linear(128, 10)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(.2)

        # self.flatten = nn.Flatten()

    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)

        #print(x.shape)

        x = self.cnn_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)

        #print(x.shape)

        x = self.flatten(x)
        #print(x.shape)

        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.linear_layer_3(x)
        #logits = self.sigmoid(x)
        return x

# Model Creation

In [ ]:
model = NeuralNetwork(784)
print(model)



NeuralNetwork(
  (cnn_layer_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer_1): Linear(in_features=1568, out_features=512, bias=True)
  (linear_layer_2): Linear(in_features=512, out_features=128, bias=True)
  (linear_layer_3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [ ]:
from torchsummary import summary
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             416
           Dropout-2           [-1, 16, 28, 28]               0
              ReLU-3           [-1, 16, 28, 28]               0
         MaxPool2d-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]          12,832
           Dropout-6           [-1, 32, 14, 14]               0
              ReLU-7           [-1, 32, 14, 14]               0
         MaxPool2d-8             [-1, 32, 7, 7]               0
           Flatten-9                 [-1, 1568]               0
           Linear-10                  [-1, 512]         803,328
          Dropout-11                  [-1, 512]               0
             ReLU-12                  [-1, 512]               0
           Linear-13                  [-1, 128]          65,664
          Dropout-14                  [

# Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [ ]:
def trainModel(model, loss_fn, optimizer):
    model.train()

    batch = 100

    loss = 0
    for i in range(train.shape[0]):
      x, y = torch.reshape(train[i],(1,1,28,28)), torch.tensor([train_y[i]], dtype=torch.float)
      label=torch.zeros([1,10,], dtype=torch.float32)
      label[0,int(y.item())]=1
      # Compute prediction error
      pred = model(x)
      #print(pred)
      #print(label)
      loss += loss_fn(pred, label)

      if i>0 and (i+1)%batch == 0:
          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          print(f'Training Loss: {loss.item():.4f}', end="\r")
          loss = 0
    print()

# Model testing

In [ ]:
def testModel(model, loss_fn):
    model.eval()

    size = test.shape[0]
    correct=0
    loss = 0
    total =10000
    with torch.no_grad():
      for i in range(test.shape[0]):
        x, y = torch.reshape(test[i],(1,1,28,28)), torch.tensor([test_y[i]], dtype=torch.float)
        label=torch.zeros([1,10,], dtype=torch.float32)
        label[0,int(y.item())]=1
        pred = model(x)
        #print(pred)
        predicted = torch.argmax(pred)
        #print(predicted)
        #print(y)


        # Total correct predictions
        correct += (predicted == int(y)).sum()

        loss += loss_fn(pred, label)

    loss /= size
    accuracy = 100 * correct.item() / total

    # Print Loss
    print('Loss: {}. Accuracy: {}'.format({loss}, accuracy))


In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainModel(model, loss_fn, optimizer)
    testModel(model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Training Loss: 0.8743
Loss: {tensor(0.0794)}. Accuracy: 97.75
Epoch 2
-------------------------------
Training Loss: 0.6520
Loss: {tensor(0.0633)}. Accuracy: 98.16
Epoch 3
-------------------------------
Training Loss: 0.6977
Loss: {tensor(0.0399)}. Accuracy: 98.8
Epoch 4
-------------------------------
Training Loss: 0.1341
Loss: {tensor(0.0383)}. Accuracy: 98.8
Epoch 5
-------------------------------
Training Loss: 1.1260
Loss: {tensor(0.0332)}. Accuracy: 99.01
Done!
